## Домашнє завдання. Аналіз даних у PySpark

###  1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, bround

In [2]:
conf = SparkConf() \
    .setAppName("goit-de-hw-03") \
    .setMaster("spark://AMD:7077")

In [3]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/01/24 02:10:53 WARN Utils: Your hostname, AMD resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/01/24 02:10:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/24 02:10:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
users_fname = './data/users.csv'
purchases_fname = './data/purchases.csv'
products_fname = './data/products.csv'

In [5]:
users_df = spark.read.csv(users_fname, header=True)
purchases_df = spark.read.csv(purchases_fname, header=True)
products_df = spark.read.csv(products_fname, header=True)

In [6]:
users_df.show(3)
purchases_df.show(3)
products_df.show(3)

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
+-------+------+---+-----------------+
only showing top 3 rows

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
+-----------+-------+----------+----------+--------+
only showing top 3 rows

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
+----------+------------+-----------+-----

### 2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

In [7]:
users_df = users_df.dropna() \
    .withColumn('age', col('age').cast('int'))

purchases_df = purchases_df.dropna() \
    .withColumn('quantity', col('quantity').cast('int'))

products_df = products_df.dropna() \
    .withColumn('price', col('price').cast('float'))

In [8]:
users_df.printSchema()
purchases_df.printSchema()
products_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- email: string (nullable = true)

root
 |-- purchase_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- quantity: integer (nullable = true)

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: float (nullable = true)



### 3. Визначте загальну суму покупок за кожною категорією продуктів.

In [9]:
def profit_per_category(df):
    return df.join(
            products_df.select('category', 'price', 'product_name', 'product_id'),
            df.product_id == products_df.product_id,
            'inner'
        ) \
        .groupBy(df.product_id, 'product_name', 'category', 'price') \
        .agg(sum('quantity').alias('total_quantity')) \
        .withColumn("product_profit", col("total_quantity") * col("price")) \
        .groupBy('category') \
        .agg(sum('product_profit').alias('total_profit')) \
        .orderBy(col('total_profit').desc()) \
        .withColumn('total_profit', bround(col('total_profit'), 2))

In [10]:
print('Total profit per category: ')
profit_per_category(purchases_df).show()

Total profit per category: 
+-----------+------------+
|   category|total_profit|
+-----------+------------+
|     Sports|      1802.5|
|       Home|      1523.5|
|Electronics|      1174.8|
|   Clothing|       790.3|
|     Beauty|       459.9|
+-----------+------------+



### 4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [11]:
purchases_18_25_df = purchases_df.join(
        users_df.select('user_id', 'age'),
        purchases_df.user_id == users_df.user_id,
        'inner'
    ) \
    .where(col("age").between(18, 25)) \
    .drop('age', 'user_id', 'date') \

profit_18_25_df = profit_per_category(purchases_18_25_df)

print('Profit per category for users between 18 and 25: ')
profit_18_25_df.show()

Profit per category for users between 18 and 25: 
+-----------+------------+
|   category|total_profit|
+-----------+------------+
|       Home|       361.1|
|     Sports|       310.5|
|Electronics|       249.6|
|   Clothing|       245.0|
|     Beauty|        41.4|
+-----------+------------+



### 5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

In [12]:
overall_profit_18_25 = profit_18_25_df.agg(sum('total_profit')).collect()[0][0]

profit_percent_18_25_df = profit_18_25_df \
    .withColumn('percentage', col('total_profit') / overall_profit_18_25 * 100) \
    .withColumn('percentage', bround(col('percentage'), 2)) \
    .orderBy(col('percentage').desc()) \

print('Percentage of profit per category for users between 18 and 25: ')
profit_percent_18_25_df.show()


Percentage of profit per category for users between 18 and 25: 
+-----------+------------+----------+
|   category|total_profit|percentage|
+-----------+------------+----------+
|       Home|       361.1|      29.9|
|     Sports|       310.5|     25.71|
|Electronics|       249.6|     20.67|
|   Clothing|       245.0|     20.29|
|     Beauty|        41.4|      3.43|
+-----------+------------+----------+



### 6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [13]:
print('Percentage of profit per category for users between 18 and 25 (top 3): ')
profit_percent_18_25_df.select('category', 'percentage').show(3)

Percentage of profit per category for users between 18 and 25 (top 3): 
+-----------+----------+
|   category|percentage|
+-----------+----------+
|       Home|      29.9|
|     Sports|     25.71|
|Electronics|     20.67|
+-----------+----------+
only showing top 3 rows



In [14]:
spark.stop()